In [2]:
import pandas as pd
import numpy as np

In [3]:
data = pd.read_csv('weather_data_month.csv', sep=',').drop(['text'], axis='columns')

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11874 entries, 0 to 11873
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   января    11874 non-null  int64
 1   февраля   11874 non-null  int64
 2   марта     11874 non-null  int64
 3   апреля    11874 non-null  int64
 4   мая       11874 non-null  int64
 5   июня      11874 non-null  int64
 6   июля      11874 non-null  int64
 7   августа   11874 non-null  int64
 8   сентября  11874 non-null  int64
 9   октября   11874 non-null  int64
 10  ноября    11874 non-null  int64
 11  декабря   11874 non-null  int64
 12  month     11874 non-null  int64
dtypes: int64(13)
memory usage: 1.2 MB


In [5]:
X = data.drop('month', axis=1)
y = data['month'].astype('int64')

In [6]:
X.shape, y.shape

((11874, 12), (11874,))

In [7]:
from sklearn.tree import DecisionTreeClassifier

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, 
                                                      test_size=0.3, 
                                                      random_state=42)

In [9]:
dtc = DecisionTreeClassifier(random_state=17, max_depth=12 )

In [10]:
from sklearn.model_selection import  cross_val_score
np.mean(cross_val_score(dtc, X_train, y_train, cv=5))

0.9800266723976865

In [11]:
from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.metrics import accuracy_score
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [12]:
results = cross_val_score(dtc, X, y, cv=skf)

In [13]:
print("CV accuracy score: {:.2f}%".format(results.mean()*100))

CV accuracy score: 98.03%


In [14]:
dtc.fit(X, y)

DecisionTreeClassifier(max_depth=12, random_state=17)

In [15]:
import csv

In [16]:
from utils import BagOfWords

In [22]:
with open('test_data.csv', encoding='utf-8', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        text = row['text']
        b = BagOfWords(text)
        print(text)
        print(dtc.predict([b.months_bag]))
        print('---------')

Желтый уровень опасности. 8 сентября (вторник) в отдельных районах ночью по восточной половине
[9]
---------
Желтый уровень опасности. 7 сентября (понедельник) в отдельных районах республики ожидаются грозы
[9]
---------
Желтый уровень опасности. 6 сентября (воскресенье) в отдельных районах республики ожидаются грозы.
[9]
---------
Желтый уровень опасности. Вечером 5 сентября (суббота) местами по западу республики ожидаются грозы.
[9]
---------
Желтый уровень опасности. 5 сентября (суббота) ночью и утром в отдельных районах республики ожидается туман
[9]
---------
Желтый уровень опасности. Днем 4 сентября (пятница) местами по востоку республики ожидаются грозы.
[9]
---------
Оранжевый уровень опасности. Сегодня
[8]
---------
Желтый уровень опасности. 4 сентября (пятница) местами по восточной половине республики ожидаются грозы.
[9]
---------
Желтый уровень опасности. 3 сентября (четверг) местами в центральных и восточных районах страны ожидаются грозы.
[9]
---------
Оранжевый уровень о

Оранжевый уровень опасности. 4 июля (суббота) по юго-востоку республики во многих районах ожидаются грозы
[7]
---------
Оранжевый уровень опасности. Утром и днем 3 июля (пятница) во многих районах республики ожидаются грозы. В дневные часы при грозах местами шквалистое усиление ветра порывами до 15-20 м/с
[7]
---------
Оранжевый уровень опасности. Сегодня
[8]
---------
Желтый уровень опасности. 2 июля (четверг) в отдельных районах республики ожидаются грозы. Днем местами сильные дожди
[7]
---------
Оранжевый уровень опасности. 30 июня (вторник) ночью по западной половине
[6]
---------
Желтый уровень опасности. ВНИМАНИЕ! 29 июня (понедельник) местами по республике ожидаются грозы. В дневные часы в отдельных районах страны максимальная температура воздуха достигнет +30&#176;С.
[6]
---------
Оранжевый уровень опасности. В период с 9.00 до 15.00 28 июня (воскресенье) на большей части территории республики ожидаются грозы. При грозах местами сильные ливни
[6]
---------
Оранжевый уровень опа

[5]
---------
Желтый уровень опасности. Днем 7 мая (четверг) местами по восточной половине Беларуси ожидаются сильные дожди.
[5]
---------
Желтый уровень опасности. 7 мая (четверг) в отдельных районах республики ожидаются сильные дожди.
[5]
---------
Желтый уровень опасности. Днем 6 мая (среда) местами по республике ожидаются сильные дожди
[5]
---------
Желтый уровень опасности. 6 мая (среда) местами по республике ожидаются сильные дожди
[5]
---------
Желтый уровень опасности. 5 мая (вторник) ночью в отдельных районах по юго-востоку
[5]
---------
Желтый уровень опасности. Днем 4 мая (понедельник) местами по востоку страны ожидаются грозы
[5]
---------
Желтый уровень опасности. 4 мая (понедельник) местами по восточной половине страны ожидаются грозы
[5]
---------
Оранжевый уровень опасности. Сегодня
[8]
---------
Желтый уровень опасности. 3 мая (воскресенье) в отдельных районах республики ожидаются сильные дожди и грозы.
[5]
---------
Желтый уровень опасности. 2 мая (суббота) в отдельны